Mount google drive

In [1]:
import os
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Utility classes

In [9]:
import sys
import tensorflow as tf

class Vector:
  def __init__(self, x, y, z):
    self.x = x
    self.y = y
    self.z = z

  def __add__(self, other):
    return Vector(self.x + other.x, self.y + other.y, self.z + other.z)

  def __sub__(self, other):
    return Vector(self.x - other.x, self.y - other.y, self.z - other.z)

  def __div__(self, divisor):
    return Vector(self.x/divisor, self.y/divisor, self.z/divisor)

  def __mul__(self, multiplier):
    return Vector(self.x*multiplier, self.y*multiplier, self.z*multiplier)

class BoundingBox:
  def __init__(self):
    min_float = sys.float_info.min
    max_float = sys.float_info.max
    self.min = Vector(max_float, max_float, max_float)
    self.max = Vector(min_float, min_float, min_float)


ObjToVolumetricMeshConverter

In [38]:
import numpy as np
import shutil
import zlib

class ObjToVolumetricMeshConverter:

  def Convert(self, src_folder_path, dest_folder_path, clamp_range, grid_dimensions):
    point_cloud_set = self.ProcessOBJFolder(src_folder_path)
    point_cloud_set = self.ClampInRange(clamp_range, point_cloud_set)
    voxelized_mesh_set = self.ConvertToVoxelizedMeshSet(point_cloud_set, grid_dimensions)
    self.SaveOnDisk(voxelized_mesh_set, dest_folder_path)

  def SaveOnDisk(self, voxelized_mesh_set, dest_folder_path):
    i=0
    for mesh in voxelized_mesh_set:
      filename = dest_folder_path+str(i)
      self.SaveArray(mesh, filename)
      i = i+1

  def SaveArray(self, array, filename):
    np.savez_compressed(filename, array=array)
    os.rename(filename + ".npz", filename + ".npy.z")

  def ProcessOBJFolder(self, folder_path):
    point_cloud_set = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            file_path = os.path.join(root, file)
            point_cloud_set.append(self.ParseOBJFile(file_path))
    print(len(point_cloud_set))
    return point_cloud_set

  def ParseOBJFile(self, file_path):
    point_cloud = []
    with open(file_path, 'r') as f:
        for line in f:
            if line.startswith('v '):
                line_split = line.split()[1:]
                point = Vector(float(line_split[0]), float(line_split[1]), float(line_split[2]))
                point_cloud.append(point)
    return point_cloud

  def GenerateBoundingBox(self, point_cloud):
    bounding_box = BoundingBox()
    for v in point_cloud:
      bounding_box.min.x = min(v.x, bounding_box.min.x)
      bounding_box.min.y = min(v.y, bounding_box.min.y)
      bounding_box.min.z = min(v.z, bounding_box.min.z)
      bounding_box.max.x = max(v.x, bounding_box.max.x)
      bounding_box.max.y = max(v.y, bounding_box.max.y)
      bounding_box.max.z = max(v.z, bounding_box.max.z)
    return bounding_box

  def ClampInRange(self, clamp_range, point_cloud_set):
    for i in range(0, len(point_cloud_set)):
      bounding_box = self.GenerateBoundingBox(point_cloud_set[i])

      orig_scale_x = bounding_box.max.x - bounding_box.min.x
      orig_scale_y = bounding_box.max.y - bounding_box.min.y
      orig_scale_z = bounding_box.max.z - bounding_box.min.z

      center_point = (bounding_box.max + bounding_box.min)*0.5

      centered_points = [Vector(v.x - center_point.x,
                                v.y - center_point.y,
                                v.z - center_point.z)
                          for v in point_cloud_set[i]]

      scale_factor_x = (clamp_range.max.x - clamp_range.min.x) / orig_scale_x
      scale_factor_y = (clamp_range.max.y - clamp_range.min.y) / orig_scale_y
      scale_factor_z = (clamp_range.max.z - clamp_range.min.z) / orig_scale_z

      point_cloud_set[i] = [Vector(v.x * scale_factor_x, v.y * scale_factor_y, v.z * scale_factor_z) for v in centered_points]
    return point_cloud_set

  def ConvertToVoxelizedMeshSet(self, point_cloud_set, grid_dimensions):
    voxelized_mesh_set = []
    for point_cloud in point_cloud_set:
      voxelized_mesh = np.array(self.ConvertToVoxelizedMesh(point_cloud, grid_dimensions))
      voxelized_mesh = voxelized_mesh.reshape((grid_dimensions.x, grid_dimensions.y, grid_dimensions.z))
      voxelized_mesh_set.append(voxelized_mesh)
    return voxelized_mesh_set

  def ConvertToVoxelizedMesh(self, point_cloud, grid_dimensions):
    size = grid_dimensions.x*grid_dimensions.y*grid_dimensions.z
    voxelized_mesh = [0] * size
    bounding_box = self.GenerateBoundingBox(point_cloud)
    for point in point_cloud:
      voxel_x = int((point.x - bounding_box.min.x) / (bounding_box.max.x - bounding_box.min.x) * grid_dimensions.x)
      voxel_y = int((point.y - bounding_box.min.y) / (bounding_box.max.y - bounding_box.min.y) * grid_dimensions.y)
      voxel_z = int((point.z - bounding_box.min.z) / (bounding_box.max.z - bounding_box.min.z) * grid_dimensions.z)
      if 0 <= voxel_x < grid_dimensions.x and 0 <= voxel_y < grid_dimensions.y and 0 <= voxel_z < grid_dimensions.z:
        voxel_index = voxel_z * (grid_dimensions.x * grid_dimensions.y) + voxel_y * grid_dimensions.x + voxel_x
        voxelized_mesh[voxel_index] = 1
    return voxelized_mesh

Usage

In [39]:
import zipfile
zip_file_path = '/content/drive/MyDrive/SimJEB_surfmesh_(obj).zip'
src_folder_path = '/content/drive/MyDrive/SimJEB_surfmesh'

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(src_folder_path)

clamp_range = BoundingBox()
clamp_range.min = Vector(-5,-5,-5)
clamp_range.max = Vector(5,5,5)
grid_dimensions = Vector(32,32,32)
dest_folder_path = "/content/drive/MyDrive/dataset_2/"

objToVolumetricMeshConverter = ObjToVolumetricMeshConverter()
objToVolumetricMeshConverter.Convert(src_folder_path, dest_folder_path, clamp_range, grid_dimensions)

382


Point Cloud Visualizer

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

class PointCloudVisualizer:
    def __init__(self):
        self.fig = plt.figure()
        self.ax = self.fig.add_subplot(111, projection='3d')

    def Visualize(self, point_cloud_data):
        num_points = len(point_cloud_data)

        x = [point[0] for point in point_cloud_data]
        y = [point[1] for point in point_cloud_data]
        z = [point[2] for point in point_cloud_data]

        self.ax.scatter(x, y, z)
        self.ax.set_xlabel('X')
        self.ax.set_ylabel('Y')
        self.ax.set_zlabel('Z')

        plt.show()